In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image


In [ ]:
class EyeTrackingDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.X = np.load(f'{data_path}/eye_images.npy')
        self.y = np.load(f'{data_path}/gaze_coordinates.npy')
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create datasets
train_dataset = EyeTrackingDataset('path/to/training/data', transform=transform)
test_dataset = EyeTrackingDataset('path/to/test/data', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class EyeTrackingCNN(nn.Module):
    def __init__(self):
        super(EyeTrackingCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 64)  # Adjust this based on your input size
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 64 * 5 * 5)  # Adjust this based on your input size
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model
model = EyeTrackingCNN()
print(model)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# Train the model
train_model(model, train_loader, criterion, optimizer)

In [ ]:
def evaluate_model(model, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    total_mae = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            mae = torch.abs(outputs - labels).mean()
            total_mae += mae.item()

    avg_mae = total_mae / len(test_loader)
    print(f"Test MAE: {avg_mae}")

# Evaluate the model
evaluate_model(model, test_loader)

In [ ]:
torch.save(model.state_dict(), 'eye_tracking_model.pth')